# Imports

In [1]:
import tensorflow.python.keras.backend as K

In [6]:
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn

mpl.rc("axes", labelsize=14)
mpl.rc("xtick", labelsize=12)
mpl.rc("ytick", labelsize=12)
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

In [7]:
import skimage.io
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Flatten, Dropout,BatchNormalization ,Activation
from tensorflow.keras.models import Model, Sequential
from keras.applications.nasnet import NASNetLarge
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam

# Construção Modelo

In [8]:
from tensorflow.python.keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, InceptionResNetV2
from tensorflow.python.keras import regularizers
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax
from tensorflow.keras.layers import Dense, Flatten, Dropout,BatchNormalization ,Activation
import cv2

In [9]:
IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS, NUM_CLASSES = 48, 48, 1, 7
base_model = tf.keras.applications.EfficientNetB0(input_shape=(48,48,3),include_top=False,weights="imagenet")

In [10]:
# Freezing Layers
for layer in base_model.layers:
    layer.trainable=False

In [11]:
model=Sequential()
model.add(base_model)
model.add(Dropout(0.5))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(7,activation='softmax'))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, 2, 2, 1280)        4049571   
_________________________________________________________________
dropout (Dropout)            (None, 2, 2, 1280)        0         
_________________________________________________________________
flatten (Flatten)            (None, 5120)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 5120)              20480     
_________________________________________________________________
dense (Dense)                (None, 32)                163872    
_________________________________________________________________
batch_normalization_1 (Batch (None, 32)                128       
_________________________________________________________________
activation (Activation)      (None, 32)                0

In [13]:
def f1_score(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [14]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),  
      tf.keras.metrics.AUC(name='auc'),
        f1_score,
]

In [15]:
lrd = ReduceLROnPlateau(monitor = 'val_loss',patience = 20,verbose = 1,factor = 0.50, min_lr = 1e-10)

mcp = ModelCheckpoint('model.h5')

es = EarlyStopping(verbose=1, patience=20)

In [16]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=METRICS)

# Treinamento e teste 

In [17]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   validation_split = 0.2,
                                  
        rotation_range=5,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        #zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

# valid_datagen = ImageDataGenerator(rescale = 1./255,
#                                   validation_split = 0.2)

test_datagen  = ImageDataGenerator(rescale = 1./255
                                  )

In [18]:
train_dataset  = train_datagen.flow_from_directory(directory = 'train_apagar',
                                                   target_size = (48,48),
                                                   color_mode = "rgb",
                                                   class_mode = 'categorical',
                                                   subset = 'training',
                                                   batch_size = 64,
                                                   shuffle=True)

Found 119 images belonging to 7 classes.


In [19]:
teste_y = train_dataset.classes

In [20]:
###### valid_datagen OU train_datagen?????/
valid_dataset = train_datagen.flow_from_directory(directory = 'train_apagar',
                                                  target_size = (48,48),
                                                  color_mode = "rgb",
                                                  class_mode = 'categorical',
                                                  subset = 'validation',
                                                  batch_size = 64,
                                                  shuffle=True)

Found 28 images belonging to 7 classes.


In [21]:
test_dataset = test_datagen.flow_from_directory(directory = 'test_apagar',
                                                  target_size = (48,48),
                                                  color_mode = "rgb",
                                                  class_mode = 'categorical',
                                                  batch_size = 64,
                                                  shuffle=True)

Found 98 images belonging to 7 classes.


# Fit do modelo 

In [22]:
from keras.callbacks import CSVLogger

In [23]:
epochs = 60
batch_size = 64
history = model.fit(x = train_dataset,epochs = epochs,validation_data = valid_dataset)

Epoch 1/60
2/2 [==============================] - 7s 2s/step - loss: 2.3484 - accuracy: 0.8451 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.4816 - f1_score: 0.0000e+00 - val_loss: 2.0000 - val_accuracy: 0.8571 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5000 - val_f1_score: 0.0000e+00
Epoch 2/60
2/2 [==============================] - 0s 213ms/step - loss: 2.4324 - accuracy: 0.8463 - precision: 0.0909 - recall: 0.0084 - auc: 0.4475 - f1_score: 0.0169 - val_loss: 1.9859 - val_accuracy: 0.8571 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5000 - val_f1_score: 0.0000e+00
Epoch 3/60
2/2 [==============================] - 0s 222ms/step - loss: 2.2323 - accuracy: 0.8499 - precision: 0.2000 - recall: 0.0168 - auc: 0.5257 - f1_score: 0.0312 - val_loss: 1.9748 - val_accuracy: 0.8571 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5004 - val_f1_score: 0.0000e+00
Epoch 4/60
2/2 [==============================] - 0s 191ms/step - l

2/2 [==============================] - 1s 358ms/step - loss: 2.0897 - accuracy: 0.8523 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5164 - f1_score: 0.0000e+00 - val_loss: 1.9509 - val_accuracy: 0.8571 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5000 - val_f1_score: 0.0000e+00
Epoch 28/60
2/2 [==============================] - 1s 419ms/step - loss: 2.0386 - accuracy: 0.8535 - precision: 0.2000 - recall: 0.0084 - auc: 0.5490 - f1_score: 0.0169 - val_loss: 1.9514 - val_accuracy: 0.8571 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.4940 - val_f1_score: 0.0000e+00
Epoch 29/60
2/2 [==============================] - 1s 481ms/step - loss: 2.1894 - accuracy: 0.8523 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.4454 - f1_score: 0.0000e+00 - val_loss: 1.9516 - val_accuracy: 0.8571 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5000 - val_f1_score: 0.0000e+00
Epoch 30/60
2/2 [==============================] - 1s 515ms/s

Epoch 53/60
2/2 [==============================] - 1s 349ms/step - loss: 2.0440 - accuracy: 0.8547 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5186 - f1_score: 0.0000e+00 - val_loss: 1.9532 - val_accuracy: 0.8571 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5000 - val_f1_score: 0.0000e+00
Epoch 54/60
2/2 [==============================] - 1s 364ms/step - loss: 2.0548 - accuracy: 0.8559 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5131 - f1_score: 0.0000e+00 - val_loss: 1.9534 - val_accuracy: 0.8571 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.4966 - val_f1_score: 0.0000e+00
Epoch 55/60
2/2 [==============================] - 1s 407ms/step - loss: 2.0408 - accuracy: 0.8547 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.4976 - f1_score: 0.0000e+00 - val_loss: 1.9542 - val_accuracy: 0.8571 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.4983 - val_f1_score: 0.0000e+00
Epoch 56/60
2/2 [====================

## Salvando o history em json

In [ ]:
hist_df = pd.DataFrame(history.history) 

# save to json:  
hist_json_file = 'history_RESNET50.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

## Salvando o modelo

In [ ]:
model.save("modelo_RESNET50")

In [ ]:
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
train_l = history.history['val_accuracy']
fig.set_size_inches(12,4)

ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('Training Accuracy vs Validation Accuracy')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].legend(['Train', 'Validation'], loc='upper left')

ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('Training Loss vs Validation Loss')
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Epoch')
ax[1].legend(['Train', 'Validation'], loc='upper left')

plt.show()

## Evaluate 

In [ ]:
results = model.evaluate(test_dataset, batch_size=64)